In [6]:
# 필요한 모듈 임포트
import os
from dotenv import load_dotenv
import httpx

class CustomHTTPClient(httpx.Client):
    def __init__(self, *args, **kwargs):
        kwargs.pop("proxies", None) # Remove the 'proxies' argument if present
        super().__init__(*args, **kwargs)

from openai import OpenAI

#환경변수 취득
load_dotenv()
#OpenAI API 클라이언트 생성
client = OpenAI(api_key=os.environ['API_KEY'], http_client=CustomHTTPClient())

# 모델명
model_name = "gpt-3.5-turbo-instruct"

In [13]:
# 프롬프트 설정
prompt = "언어 모델을 사용할 때 포인트는 뭘까?"
# suffix = "라고 생각합니다."

# API 리퀘스트
response = client.completions.create(
    model = model_name, 
    prompt = prompt,
    max_tokens = 50,
    n = 2,
    best_of=4
)

# 결과 표시
for choice in response.choices:
    print("-" * 20)
    print(choice.text.strip())

print(response.usage)

--------------------
1. 데이터의 크기와 다양성: 언어 모델을 학습시키기 위해서는 대규모의 다양한 데이터가 필요하다. 따
--------------------
1. 데이터의 품질: 언어 모델의 성능은 데이터의 품질에 크게 영향을 받는다. 따라서 데이터를 수
CompletionUsage(completion_tokens=200, prompt_tokens=20, total_tokens=220, completion_tokens_details=None, prompt_tokens_details=None)


In [36]:
# 프롬프트 설정
prompt = "폐암 환자에게 도움이 되는 간식을 알고 싶어"

# API 리퀘스트
response = client.completions.create(
    model = model_name, 
    prompt = prompt,
    max_tokens = 100,
    logprobs = 3
)

print(response.choices[0].text.strip())

요!
1. 고구마칩: 고구마는 항산화 작용이 강하고 비타민 A, C, E가 풍부하여 면역력 증강에 도움이 됩니다. 또한 건강한 치즈와 함께 섭취하면 단백질과 칼슘


In [25]:
import tiktoken
from tiktoken.core import Encoding

# OpenAI APIの特定のモデルに対応するトークナイザーを取得
encoding: Encoding = tiktoken.encoding_for_model(model_name)

In [26]:
# 回答をトークンごとに確認
for token in response.choices[0].logprobs.tokens:
    print("[", token.replace("\n",""), "]", end=" ")

[ 해 ] [ 요 ] [  ] [  ] [ 1 ] [ . ] [ bytes: \xea\xb3 ] [ bytes:\xa0 ] [ 구 ] [ bytes:\xeb\xa7 ] [ bytes:\x88 ] [ bytes: \xec\x8a ] [ bytes:\xac ] [ 라 ] [ 이 ] [ 스 ] [  - ] [ bytes: \xea\xb3 ] [ bytes:\xa0 ] [ 구 ] [ bytes:\xeb\xa7 ] [ bytes:\x88 ] [ 는 ] [  비 ] [ bytes:\xed\x83 ] [ bytes:\x80 ] [ bytes:\xeb\xaf ] [ bytes:\xbc ] [ A ] [ , ] [ bytes: \xec\xb9 ] [ bytes:\xbc ] [ bytes:\xeb\xa5 ] [ bytes:\xa8 ] [ , ] [ bytes: \xec\x8b ] [ bytes:\x9d ] [ 이 ] [ bytes:\xec\x84 ] [ bytes:\xac ] [ bytes:\xec\x9c ] [ bytes:\xa0 ] [  등 ] [ 이 ] [ bytes: \xed ] [ bytes:\x92 ] [ bytes:\x8d ] [ 부 ] [ 하 ] [ bytes:\xeb\xa9 ] [ bytes:\xb0 ] [ bytes: \xed\x98 ] [ bytes:\x88 ] [ bytes:\xec\x95 ] [ bytes:\xa1 ] [ bytes:\xec ] [ bytes:\x88 ] [ bytes:\x9c ] [ 환 ] [ 을 ] [ bytes: \xeb\x8f ] [ bytes:\x95 ] [ 고 ] [ bytes: \xeb\xa9 ] [ bytes:\xb4 ] [ bytes:\xec\x97 ] [ bytes:\xad ] [ 력 ] [ 을 ] [ bytes: \xea\xb0 ] [ bytes:\x95 ] [ 화 ] [ 해 ] [ bytes:\xec\xa4 ] [ bytes:\x8d ] [ 니다 ] [ . ] [ 2 ] [ . ] [ bytes: \xeb ] [ by

In [27]:
# 選択されたトークンの確率を表示
response.choices[0].logprobs.token_logprobs[:5]

[-3.303797, -0.1506058, -2.3085585, -0.9140037, -1.1073347]

In [28]:
# トークン上位logprobs個の確率を表示
response.choices[0].logprobs.top_logprobs[:5]

[{'요': -1.1779463, '서': -1.4530146, '합니다': -2.7171843},
 {'요': -0.1506058, '\n\n': -3.12717, '\n': -4.141071},
 {'\n\n': -1.0303297, '.': -1.9529123, '.\n\n': -2.161376},
 {'\n': -0.9140037, '1': -1.2380717, 'bytes:\\xec\\x95': -2.2539656},
 {'간': -1.0186335, '1': -1.1073347, 'bytes:\\xec\\x95': -2.1882157}]